<a href="https://colab.research.google.com/github/Asritha0606/GenAIcohort_May2025/blob/main/GenAIcohort_May2025_RAGAssignment_Asritha.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Install Required Packages

In [1]:
!pip install -q langchain faiss-cpu gradio sentence-transformers groq langchain-groq langchain-community text-generation

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.3/31.3 MB 10.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 130.2/130.2 kB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 19.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 438.1/438.1 kB 14.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.0/363.0 kB 12.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.4/44.4 kB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 2.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 28.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 36.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 19.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5

In [2]:
from langchain.vectorstores import FAISS
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.document_loaders import TextLoader
from langchain.chains import ConversationalRetrievalChain
from langchain.memory import ConversationBufferWindowMemory
from langchain_groq import ChatGroq
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain.chains import create_retrieval_chain, create_history_aware_retriever
from langchain.prompts import PromptTemplate
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain.memory import ConversationBufferWindowMemory
import gradio as gr
import re

RAG + LLM + Memory + Guardrails + Cache

Load and split document

In [3]:
loader = TextLoader("/content/AYUSHMAN BHARAT.txt")
docs = loader.load()

In [4]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=100)
chunks = text_splitter.split_documents(docs)

Create embeddings and vector store

In [5]:
embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
vectorstore = FAISS.from_documents(chunks, embeddings)
retriever = vectorstore.as_retriever(search_kwargs={"k": 3})

<ipython-input-5-805027492>:1: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.5k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [6]:
import os
from google.colab import userdata
groq_api_key = userdata.get('groq_api')
os.environ["GROQ_API_KEY"] = groq_api_key

In [7]:
llm = ChatGroq(model="llama-3.3-70b-versatile", temperature=0)

Memory for 3-turn history

In [8]:
memory = ConversationBufferWindowMemory(k=3, return_messages=True)

<ipython-input-8-2011558429>:1: LangChainDeprecationWarning: Please see the migration guide at: https://python.langchain.com/docs/versions/migrating_memory/
  memory = ConversationBufferWindowMemory(k=3, return_messages=True)


RAG QA Chain

In [9]:

# Step 1: Question rephrasing prompt
question_prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a helpful assistant for answering questions about Ayushman Bharat."),
    MessagesPlaceholder(variable_name="chat_history"),
    ("human", "{input}"),
    ("human", "Given the above conversation, generate a search query for relevant information:")
])

# Step 2: History-aware retriever (rephrases the query using memory)
retriever_with_context = create_history_aware_retriever(
    llm=llm,
    retriever=retriever,
    prompt=question_prompt
)

# Step 3:  Include {context} in the answer generation prompt
qa_prompt = ChatPromptTemplate.from_messages([
    ("system", "You are an expert assistant answering public queries about Ayushman Bharat using the following context:\n\n{context}"),
    MessagesPlaceholder(variable_name="chat_history"),
    ("human", "{input}")
])

# Step 4: Document answer chain
document_chain = create_stuff_documents_chain(
    llm=llm,
    prompt=qa_prompt
)

# Step 5: Final RAG chain
qa_chain = create_retrieval_chain(
    retriever=retriever_with_context,
    combine_docs_chain=document_chain
)


Domain guardrails

In [10]:
domain_keywords = [
    "ayushman bharat", "ayushman", "bharat",
    "health", "healthcare", "hospitals",
    "nhm", "pmjay", "hwc", "insurance"
]

def is_in_domain(query: str) -> bool:
    query = query.lower()
    return any(keyword in query for keyword in domain_keywords)


In-memory cache

In [11]:
query_cache = {}

def chat_interface(query, chat_history):
    print("Incoming query:", query)
    print("Chat history:", chat_history)

    if not is_in_domain(query):
        return chat_history + [[query, "❌ Sorry, I can only answer questions about Ayushman Bharat and healthcare topics."]]

    if query in query_cache:
        return chat_history + [[query, f"✅ Cached: {query_cache[query]}"]]

    # ✅ Correct input format for create_retrieval_chain()
    result = qa_chain.invoke({
        "input": query,
        "chat_history": memory.chat_memory.messages  # this is what the chain uses internally
    })

    answer = result["answer"]
    query_cache[query] = answer
    return chat_history + [[query, answer]]


Launch Gradio Chat UI

In [12]:
with gr.Blocks() as demo:
    gr.Markdown("### 🤖 Ayushman Bharat FAQ Chatbot")
    chatbot = gr.Chatbot()
    msg = gr.Textbox(placeholder="Ask about Ayushman Bharat...")
    clear = gr.Button("Clear")

    state = gr.State([])

    def user_interaction(user_message, chat_history):
        return chat_interface(user_message, chat_history)

    msg.submit(user_interaction, [msg, state], [chatbot])
    msg.submit(lambda: "", None, msg)
    clear.click(lambda: [], None, chatbot)



<ipython-input-12-3761753968>:3: UserWarning: You have not specified a value for the `type` parameter. Defaulting to the 'tuples' format for chatbot messages, but this is deprecated and will be removed in a future version of Gradio. Please set type='messages' instead, which uses openai-style dictionaries with 'role' and 'content' keys.
  chatbot = gr.Chatbot()


In [13]:
demo.launch(debug=True)

It looks like you are running Gradio on a hosted a Jupyter notebook. For the Gradio app to work, sharing must be enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
* Running on public URL: https://cfdf5c0f941c39acdb.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


Incoming query: What is Ayushman Bharath?
Chat history: []
Incoming query: Tell me about Elon Musk
Chat history: []
Incoming query: Who benefits from PMJAY?
Chat history: []
Incoming query: What is Ayushman Bharath?
Chat history: []
Keyboard interruption in main thread... closing server.
Killing tunnel 127.0.0.1:7860 <> https://cfdf5c0f941c39acdb.gradio.live
